In [89]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from time import time
from copy import copy
import pprint
import datetime
pp = pprint.PrettyPrinter(indent=4)
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

In [188]:
old_index = "x1t-rucio-events-2017"
new_index = "x1t-rucio-2017-beginning-test"

In [182]:
my_query = {
    "stored_fields" : ["_source"],
    "query": {
       "match_all": {}
   }
}

In [191]:
scroll = scan(es, query=my_query, index=old_index, scroll='5m', timeout="5m", size=10)

In [135]:
test_time = "2016-07-11 20:27:05"

In [136]:
datetime.datetime.strptime(test_time, "%Y-%m-%d %H:%M:%S")

datetime.datetime(2016, 7, 11, 20, 27, 5)

In [192]:
count = 0
mapping_old_new = {
    "type": "event_type"
}

for res in scroll:
    new_record = {}
    if count%10000==0: print('done: ', count)
    print(res)
    if count == 10: break
#     print(type(res))
    if "created_at" not in res["_source"].keys():
        if "created_at" in res["_source"]["payload"].keys():
#             print(res["_source"]["payload"]["created_at"])
            res["_source"]["created_at"] = res["_source"]["payload"]["created_at"]
        else:
            print("-----------------")
            print(res)
            print("-------******----------")
            continue
    if "event_type" not in res["_source"]["payload"] and "event_type" not in res["_source"]:
        print("====================")
        print(res)
        print("====================")
        continue
    elif "event_type" in res["_source"]["payload"]:
        print("==========$$$$$$$$$==========")
        print(res)
        print("==========$$$$$$$$$==========")
        res["_source"]["event_type"] = res["_source"]["payload"]["event_type"]
            #     if "created_at" not in res["_source"]["payload"].keys():
#         raise RuntimeError()
    for f, k in res['_source'].items():
#         #print('key: ',f,'  val: ', k)
        if f == "payload":                # removing "bytes" as these are not needed.
            k.pop("bytes",0)
#             try:
#                 datetime.datetime.strptime(k["created_at"], "%Y-%m-%d %H:%M:%S")
#             except:
#                 print("=========**********===========")
#                 print(res)
#                 print("=========**********===========")
#                 raise RuntimeError()
#         if f == "type":
#             continue
#         else:
#             new_record[f] = k

        
            
#     #print('------------------------')
#     if count%10000==0:print(new_record)
#     state = es.index(index=new_index, doc_type='logs', body=new_record)
#     if state['_shards']['failed'] > 0: raise RuntimeError()
#     if count%10000==0:print(state)
    count += 1
#     if count == 1000000:break
    #print('=========================')

done:  0
{'_score': None, 'sort': [0], '_id': 'AVnOK0IQdBVA34k9uL_g', '_type': 'rucio-event', '_source': {'@version': '1', 'type': 'rucio-event', 'payload': {'file-size': 2947873305, 'bytes': 2947873305, 'scope': 'x1t_SR000_161110_2156_tpc', 'name': 'XENON1T-4485-000016000-000016999-000001000.zip', 'rse': 'CCIN2P3_USERDISK', 'url': 'gsiftp://ccdcacli022.in2p3.fr:2811/pnfs/in2p3.fr/data/xenon.biggrid.nl/rucio/x1t_SR000_161110_2156_tpc/fa/8e/XENON1T-4485-000016000-000016999-000001000.zip'}, 'created_at': '2017-01-16 21:01:55', 'event_type': 'deletion-planned', '@timestamp': '2017-01-24T01:50:58.501Z'}, '_index': 'x1t-rucio-events-2017'}
{'_score': None, 'sort': [0], '_id': 'AVl5c6vguZI8SHWOr_PT', '_type': 'rucio-event', '_source': {'@timestamp': '2017-01-07T15:02:20.368Z', '@version': '1', 'type': 'rucio-event', 'payload': {'transfer-endpoint': 'https://fts.usatlas.bnl.gov:8446', 'src-type': 'DISK', 'scope': 'x1t_SR000_161021_1153_tpc', 'created_at': '2017-01-07 15:02:19', 'transfer-id':

In [70]:
new_new_index = "x1t-rucio-20*"

In [60]:
nn_scroll = scan(es, query=my_query, index=new_new_index, scroll='5m', timeout="5m", size=10)

In [61]:
count = 0
mapping_old_new = {
    "type": "event_type"
}

for res in nn_scroll:
    count += 1
    print(res)
    pp.pprint(res)
    if count>=5: break
    

{'_score': None, 'sort': [0], '_id': 'AVsXckUE0PrmlpbcOtWk', '_type': 'logs', '_source': {'@timestamp': '2017-03-29T00:23:35.809Z', 'created_at': '2017-03-29 00:23:34', '@version': '1', 'payload': {'rule-id': 'c2a80bc1d76745b899779da539f116d9', 'checksum-adler': '90a20aae', 'scope': 'x1t_SR000_161114_1612_tpc', 'dst-rse-id': '6a4d10292bd24af5b005962caf6f9984', 'request-type': 'transfer', 'file-size': 563973940, 'request-id': '5fb5643da0984c4f94fb7d30c2defc3d', 'state': 'QUEUED', 'activity': 'User Subscriptions', 'bytes': 563973940, 'checksum-md5': None, 'dst-rse': 'UC_OSG_USERDISK', 'queued_at': '2017-03-29 00:23:34.204548', 'retry-count': 1, 'name': 'XENON1T-4591-000026000-000026999-000001000.zip'}, 'event_type': 'transfer-queued'}, '_index': 'x1t-rucio-2017'}
{   '_id': 'AVsXckUE0PrmlpbcOtWk',
    '_index': 'x1t-rucio-2017',
    '_score': None,
    '_source': {   '@timestamp': '2017-03-29T00:23:35.809Z',
                   '@version': '1',
                   'created_at': '2017-03-29

In [10]:
my_query_time = {"range" :
                 {"@timestamp" :
                  {"lt" : "2017-03-01T12:00:00"} 
                 } 
                }  

In [11]:
nn_scroll = scan(es, query=my_query_time, index=new_new_index, scroll='5m', timeout="5m", size=1000)

NameError: name 'new_new_index' is not defined